# Dataset : structure analysis

## Goals

- understand the structure of Dataset object
- introduce methods for managing structure


-----

## Example

The reuse Dataset is :

<img src="https://loco-philippe.github.io/ES/ilist_merge.png" width="700">


In [1]:
from observation import Sdataset
from pprint import pprint

img = Sdataset.from_file('score.il')                # reuse Dataset from aggregation Notebook

## relationship

the relationships between Field can be shown with the img Dataset

<img src="https://loco-philippe.github.io/ES/ilist_canonical.png" width="600">


In [2]:
pprint(img.category)
print('\n', img.tree())

AttributeError: 'Sdataset' object has no attribute 'category'

## Transformation to complete Dataset

When primary Field are crossed, a Dataset is complete (can be converted into matrix or in file with low size).

In [ ]:
img.full(fillvalue=float('nan'), idxname=['full name', 'course', 'examen'])   # convert secondary indexes in primary indexes

pprint(img.category)

print('\nlength :', len(img), 'dimension :', img.dimension)

## Indexed matrix

With codec or values ajustement, a Dataset can be transformed in a matrix with choosen dimension

In [ ]:
print(img.to_xarray())
print('\nObject img is complete and have the canonical order ? ', img.complete, img.iscanonorder())


In [ ]:
img.nindex('score').tostdcodec(inplace=True)
print('absolute keys is not necessary in json object when Dataset is complete :\n')   
pprint(img.to_ntv().to_obj(), width=200)
print('\nconversion is reversible ? ', img.from_ntv(img.to_ntv()) == img)

In [ ]:
# matrix with dimension 2
img.nindex('course').coupling(img.nindex('examen'))   # transform two linked Field in two derived or coupled Field
print('new dimension : ', img.dimension, '\n')
img.to_xarray()